In [30]:
import pylab
from refine import *
import project

In [31]:
project.setup()

In [32]:
def confined_gradient_descent(
        nmw, fold_parameter=0.9, termination="growth",
        absolute_bound=float("inf"), relative_bound=7.0, etol=1, n_samples=10, max_iter=100, return_traj=False):

    """
    Performs gradient descent of a system with respect to a special confinement.

    @param nmw: system to optimize.
    @type nmw: NMSpaceWrapper
    @param fold_parameter: fold step when choosing optimal step.
    @type fold_parameter: float
    @param termination: tremination condition.
    @type termination: str
    @param absolute_bound: maximum rmsd between inital state and any intermediate state.
    @param relative_bound: maximum rmsd between actual state and the next.
    @param etol: terminates when |E(i+1) - E(i)| < etol
    @type etol: float
    @param max_iter: maximum number of iterations
    @type max_iter: int
    @param return_traj: if true all intermediate states and energies are returned. Otherwise, only final state and energy
    @type return_traj: bool
    @return: dictionary containing all the results.
        "states" - list of all states obtained.
        "values" - list of all energies obtained.
    @rtype: dict
    """

    # number of modes
    m = len(nmw.get_eigenvalues())

    # number of atoms
    n = len(nmw.get_system_position())

    # modes
    modes = nmw.get_modes()

    # special values
    r_bound_value = n * relative_bound ** 2
    a_bound_value = n * absolute_bound ** 2

    # initial state
    iteration_count = 0
    states = []
    energies = []

    anti_gradient = None

    energy_init = nmw.get_energy()

    position_init = nmw.get_position().copy()
    
    energies.append(energy_init)
    states.append(nmw.get_system_position().copy())
    # main cycle
    while True:
        # it is a weighted anti-gradient!
        anti_gradient = nmw.get_force().copy()

        # find a step
        a = np.dot(anti_gradient, anti_gradient)
#         b = -2 * np.dot(position_init, anti_gradient)
#         c = np.dot(position_init, position_init)
#         p = np.array([a, b, c - r_bound_value])
#         roots = np.roots(p)
#         upper and lower bound for the step
#         upper_bound = np.min([np.max(roots), (a_bound_value / a) ** 0.5])
#         lower_bound = np.max([0, np.min(roots)])
        upper_bound = (r_bound_value / a) ** 0.5
        lower_bound = 0
        print("bounds:", lower_bound, ";", upper_bound)  
        # find optimal step
        
        folds = np.zeros((n_samples,))
        eng = np.ones((n_samples,)) * float("inf")
        folds[0] = 1
        x1 = nmw.get_system_position().copy()
        nmw.set_position(folds[0] * upper_bound * anti_gradient + position_init)
        x2 = nmw.get_system_position().copy()
        print("rmsd init:", (np.sum((x1 - x2) ** 2) / n) ** 0.5)
        eng[0] = nmw.get_energy()
        print("energy before:", eng[0])
        for i in range(1, n_samples):
            folds[i] = folds[i - 1] * fold_parameter
            if folds[i] * upper_bound <= lower_bound:
                break
            x1 = nmw.get_system_position().copy()
            nmw.set_position(folds[i] * upper_bound * anti_gradient + position_init)
            x2 = nmw.get_system_position().copy()
            print("rmsd:", (np.sum((x1 - x2) ** 2) / n) ** 0.5)
            eng[i] = nmw.get_energy()
            print("current energy:", eng[i])
            if eng[i] < energy_init and eng[i] > eng[i - 1]:
                break
            

        # update state list
        j = np.argmin(eng)
        nmw.set_position(folds[j] * upper_bound * anti_gradient + position_init)
        states.append(nmw.get_system_position().copy())
        energies.append(eng[j])

        iteration_count += 1
        print(iteration_count, eng[j])
        if termination == "growth":
            if energy_init < eng[j]:
                states.pop()
                energies.pop()
                break
        elif termination == "etol":
            if abs(eng[j] - energy_init) < etol:
                break
        else:
            raise ValueError(f"Wrong termination type: {termination}")
        if iteration_count >= max_iter:
            break
        
        position_init = folds[j] * upper_bound * anti_gradient + position_init
        energy_init = eng[j]
        
    if not return_traj:
        return {"states": states[-1:], "energies": energies[-1:]}
    else:
        return {"states": states, "energies": energies}


In [4]:
path = str((project.data_path / "may_complex") / "1dfj.pdb")
print(path)
create_system(path, tmp_file=str(project.output_path / "tmp_system.pdb"))
t0 = time.time()
drs = DRSystem(str(project.output_path / "tmp_system.pdb"), 'charmm36.xml', refine="chain A", static="chain B")
print("construction of a system:", -t0 + time.time(), "sec")
t0 = time.time()
nmw1 = NMSpaceWrapper(drs, n_modes=10)
print("INIT ENERGY:", nmw1.get_energy())

@> 4416 atoms and 1 coordinate set(s) were parsed in 0.14s.


/home/semyon/PycharmProjects/DiplomaPython/data/may_complex/1dfj.pdb
write PDB(prody): 0.0501 sec
read PDB(openmm): 0.42836523056030273 sec


@> 8726 atoms and 1 coordinate set(s) were parsed in 0.06s.


add hydrogens and extra particles(openmm): 19.79741621017456 sec
write PDB(openmm): 0.08813929557800293 sec
1856 6870
construction of system: 10.07757830619812 sec


@> Hessian was built in 7.55s.
@> 10 modes were calculated in 15.72s.


INIT ENERGY: -10352.054227208493


In [28]:
# from time to time
t0 = time.time()
for i in range(10):
    nmw1.get_energy()
print("Energy computation takes:", (time.time() - t0) / 10, "sec")
t0 = time.time()
for i in range(10):
    nmw1.get_force()
print("Force computation takes:", (time.time() - t0) / 10, "sec")
t0 = time.time()
for i in range(10):
    nmw1.set_position(np.random.normal(0.0, 2, 10))
print("NM space position update takes:", (time.time() - t0) / 10, "sec")

Energy computation takes: 3.2929088830947877 sec
Force computation takes: 2.8933322191238404 sec
NM space position update takes: 0.10088045597076416 sec


In [29]:
# random noise
nmw1.set_position(np.random.normal(0.0, 2, 10))
print("RANDOM ENERGY:", nmw1.get_energy())

RANDOM ENERGY: -3852.5302952926577


In [12]:
nmw2 = NMSpaceWrapper(drs, n_modes=10)

@> Hessian was built in 7.56s.
@> 10 modes were calculated in 14.93s.


In [13]:
old_coords = nmw2.get_system_position().copy()
print(old_coords)

[[ 13.96779247  26.2127304  -12.14557491]
 [ 13.53106618  26.79305939 -13.21324259]
 [ 14.95767402  25.91678845 -11.0239695 ]
 ...
 [ -8.44518849  24.50321719  10.89666597]
 [ -7.18286727  23.32255275  11.39935153]
 [ -7.02878099  22.28107774  13.99978053]]


In [23]:
nmw2.set_position(np.zeros(10,))
# print(nmw2.get_energy())

In [24]:
new_coords = nmw2.get_system_position().copy()
print(np.sum((new_coords - old_coords) ** 2 / len(new_coords)) ** 0.5)

0.0


In [26]:
optimized = confined_gradient_descent(nmw2, relative_bound=0.03, fold_parameter=0.7, etol=5,
                                      n_samples=15, max_iter=30, return_traj=True, termination="etol")
print("FINAL ENERGY:", optimized["energies"])

bounds: 0 ; 0.000318497146901046
rmsd init: 0.029999999999999926
energy before: -8742.455272456027
rmsd: 0.00899999999999996
current energy: -8708.895546333202
1 -8742.455272456027
bounds: 0 ; 0.0005520446478933093
rmsd init: 0.030000000000000047
energy before: -8738.795157734181
rmsd: 0.008999999999999973
current energy: -8799.298241784754
rmsd: 0.006300000000000012
current energy: -8813.856789955556
rmsd: 0.00441000000000005
current energy: -8808.462192417464
2 -8813.856789955556
bounds: 0 ; 0.0007198568303768933
rmsd init: 0.030000000000000013
energy before: -8722.32710397405
rmsd: 0.008999999999999992
current energy: -8811.838386907468
rmsd: 0.0063000000000000295
current energy: -8846.655680600135
rmsd: 0.004410000000000028
current energy: -8854.238801863925
rmsd: 0.0030869999999999687
current energy: -8850.646859694076
3 -8854.238801863925
bounds: 0 ; 0.0008214193311427155
rmsd init: 0.029999999999999992
energy before: -8790.784275483948
rmsd: 0.008999999999999998
current energy: 

SystemError: <built-in function Context_getState> returned a result with an error set